In [1]:
# load image from aws s3 bucket
import boto3
import tensorflow as tf
from PIL import Image
import numpy as np
s3 = boto3.resource('s3')
dataset = s3.Bucket('dalle2images')
image_paths = []

bucket_name = 'dalle2images'
for obj in dataset.objects.all():
    if obj.key.endswith('.png') and obj.key.startswith('real'):
        image_paths.append(obj.key)

    elif obj.key.endswith('.png') and obj.key.startswith('fake'):
        image_paths.append(obj.key)


print("Number of images: %d" % len(image_paths))

Number of images: 42878


In [ ]:
bucket_name = 'dalle2images'
image = []
labels = []
for i in range(1,len(image_paths), 8):
    # read image from s3 bucket
    img = s3.Object(bucket_name, image_paths[i])
    img = Image.open(img.get()['Body']).resize([128,128])
    # convert image to numpy array
    img = np.array(img).astype(np.float32)
    # img = tf.convert_to_tensor(img, dtype=tf.float32)
    # add image to list
    image.append(img/255)
    if i < 21439:
      labels.append(0)
    else:
      labels.append(1)

print("Number of images: %d" % len(image))

Number of images: 5360


In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
#make labels into numpy array
alllabels = np.array(labels)
# reshape image
allimage = np.array(image).reshape(-1, 128, 128, 3)

In [ ]:
#split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(allimage, alllabels, test_size=0.2, random_state=42)

# MobileV2 model

In [ ]:
!pip install transformers

In [ ]:
!huggingface-cli login

In [ ]:
from huggingface_hub import from_pretrained_keras

reloaded_model = from_pretrained_keras('Emmawang/mobilenet_v2_fake_image_detection')

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
reloaded_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
history1 = reloaded_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
134/134 [==============================] - 17s 29ms/step - loss: 0.5634 - accuracy: 0.7460 - auc: 0.8178 - precision: 0.7434 - recall: 0.7514 - true_positives: 1611.0000 - true_negatives: 1588.0000 - false_positives: 556.0000 - false_negatives: 533.0000 - val_loss: 0.5061 - val_accuracy: 0.7845 - val_auc: 0.8735 - val_precision: 0.7905 - val_recall: 0.7743 - val_true_positives: 415.0000 - val_true_negatives: 426.0000 - val_false_positives: 110.0000 - val_false_negatives: 121.0000
Epoch 2/10
134/134 [==============================] - 2s 18ms/step - loss: 0.5594 - accuracy: 0.7523 - auc: 0.8213 - precision: 0.7464 - recall: 0.7645 - true_positives: 1639.0000 - true_negatives: 1587.0000 - false_positives: 557.0000 - false_negatives: 505.0000 - val_loss: 0.4946 - val_accuracy: 0.7948 - val_auc: 0.8736 - val_precision: 0.7599 - val_recall: 0.8619 - val_true_positives: 462.0000 - val_true_negatives: 390.0000 - val_false_positives: 146.0000 - val_false_negatives: 74.0000
Epoch 3/10

## Fine-Tuning

In [ ]:
from tensorflow.keras.optimizers import Adam
opt = Adam(learning_rate=0.0001)

reloaded_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives'])
history2 = reloaded_model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
134/134 [==============================] - 11s 41ms/step - loss: 0.5249 - accuracy: 0.7698 - auc: 0.8490 - precision: 0.7862 - recall: 0.7411 - true_positives: 1589.0000 - true_negatives: 1712.0000 - false_positives: 432.0000 - false_negatives: 555.0000 - val_loss: 0.4981 - val_accuracy: 0.7938 - val_auc: 0.8745 - val_precision: 0.7553 - val_recall: 0.8694 - val_true_positives: 466.0000 - val_true_negatives: 385.0000 - val_false_positives: 151.0000 - val_false_negatives: 70.0000
Epoch 2/10
134/134 [==============================] - 3s 21ms/step - loss: 0.5217 - accuracy: 0.7789 - auc: 0.8513 - precision: 0.7859 - recall: 0.7668 - true_positives: 1644.0000 - true_negatives: 1696.0000 - false_positives: 448.0000 - false_negatives: 500.0000 - val_loss: 0.4960 - val_accuracy: 0.7966 - val_auc: 0.8747 - val_precision: 0.7556 - val_recall: 0.8769 - val_true_positives: 470.0000 - val_true_negatives: 384.0000 - val_false_positives: 152.0000 - val_false_negatives: 66.0000
Epoch 3/10
